<a href="https://colab.research.google.com/github/sudht/NLP/blob/master/week6_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>개인 구글 드라이브와 colab 연동</h2>

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [0]:
import numpy as np
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

class FNN(object):
  
  def __init__(self, flags):
    self.learning_rate = flags["learning_rate"]  # 학습률
    self.num_classes = flags["num_classes"]      # 분류할 클래스의 개수, 여기선 숫자 분류니 10개(0~9)
    self.mode = flags["mode"]                    # 학습 or 평가 상태

    self._input_init()
    self._fnn_init()
    self._predict_init()
    self._train_init()

  # 여러 데이터 한번에 학습 -> batch (ex. batch size가 높으면 한번에 여러개 학습)
  # batch로 들어온 크기의 평균만큼 학습하고, 이런 용도로 사용, batchsize가 너무 작으면 이리저리 왔다갔다함
  # 입력 데이터, 정답 데이터, keep_prob 값을 담을 tensor 선언 (드랍아웃, 오버피팅 방지)
  def _input_init(self):
    # 입력 데이터
    self.inputs = tf.placeholder(tf.float32, [None, 28, 28], name="inputs")   # Nome은 batchsize의 위치, batch, 28 * 28
    
    
    # 정답 데이터
    self.targets = tf.placeholder(tf.int32, [None, self.num_classes], name="targets")

    # 노드를 보전할 확률
    self.keep_prob = tf.placeholder(tf.float32, [], "keep_prob")


  def _fnn_init(self):
    with tf.name_scope("fnn_layer"):
      inputs = tf.reshape(self.inputs, shape=[-1, 28*28]) # 28*28 2차원 배열을 1*(28*28)일차원 배열로 만듬
      
      # (?, 28*28) -> (?, 10)
      fully_connected_layer = tf.layers.dense(inputs=inputs, units=self.num_classes)
    
      self.outputs = fully_connected_layer    # (batch, num, classes)

  def _predict_init(self):
    # 각 클래스의 분포 값들을 softmax 함수를 이용하여 0~1사이의 값으로 변경
    self.predict_op = tf.nn.softmax(logits=self.outputs, axis=-1) # softmax는 합쳐서 1이 되도록, 확률을 나타냄. ex) [6, 3, 1] -> [0.6, 0.3, 0.1]

  def _train_init(self):
    if self.mode == "train":
      with tf.variable_scope("train_layer"):
        # 모델의 출력인 self.outputs 와 self.targets를 비교하여 loss 계산
        self.loss = tf.losses.softmax_cross_entropy(onehot_labels=self.targets, logits=self.outputs)
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.grads = self.optimizer.compute_gradients(self.loss)
        self.train_op = self.optimizer.apply_gradients(self.grads)

<h2>read_file, get_batch 함수</h2>

<pre>
<b>1. read_file(file_path)</b>
  "mnist_train.csv", "mnist_test.csv" 파일을 읽기 위한 함수
  
  read_file(file_path)
  args
    file_path : 읽고자 하는 데이터의 경로
  return
    입력 이미지에 대한 픽셀값과 해당 이미지의 라벨을 담고 있는 리스트
    
  데이터 예시)
    1, 0, 0, 0, 0, ... 0.6, 0.8, 0, 0, ... 0.7, 0.1, 0, 0, ... 0, 0, 0
    라벨, 픽셀값, 픽셀값, ... 픽셀값
</pre>  
  
  numpy array 함수 -> https://docs.scipy.org/doc/numpy/reference/generated/numpy.array.html <br>
  numpy reshape 함수 -> https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html <br>
  numpy zeros 함수 -> https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html
  

![대체 텍스트](http://nlp.kangwon.ac.kr/~nlpdemo/mnist_1.png)
<pre>
<b>2. get_batch(datas, batch_size)</b>
  전체 데이터를 batch 단위로 나누어 주기 위한 함수
  
  get_batch(datas, batch_size)
  args
    datas : 입력 이미지에 대한 픽셀값과 해당 이미지의 라벨로 이루어진 리스트
    batch_size : 한번에 학습할 데이터의 개수
  return
    batch 단위로 나뉘어진 데이터 리스트
    
  예시) 
    batch_size = 3 인 경우
  
    total_datas = [ (입력1, 라벨1), (입력2, 라벨2), (입력3, 라벨3), ... ,(입력10, 라벨10) ]
    batches = [
    [ (입력1, 입력2, 입력3), (라벨1, 라벨2, 라벨3)],
</pre>

In [0]:
import csv
import numpy as np

# 데이터를 읽고 픽셀값과 라벨을 분리하여 datas 리스트에 저장
# 픽셀값을 (28, 28) 행렬로 변환
# 라벨값은 one-hot encoding으로 변환
# 예시) 2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
def read_file(file_path):
    inFile = open(file_path, "r", encoding="utf8")
    input_file = csv.reader(inFile)
    lines = input_file

    datas = []
    ############################################################################
    # 모범답안
    for line in lines:
      data = np.array(line[1:])
      label = np.array(line[0])
      index = int(label)
      
      data = np.reshape(data, (28, 28))
      label = np.zeros(shape=(10))
      label[index] = 1
      
      datas.append((data, label))

#     내가 한것    
#     x = np.zeros(10)
#     x[lines[0]] = 1
#     datas.append(x)
#     datas.append(np.reshape(lines[1:], (28, 28)))    
    ############################################################################

    inFile.close()

    return datas

# 데이터를 batch 단위로 분할하여 저장
def get_batch(datas, batch_size):
  
  # batches : batch 단위로 저장할 리스트
  batches = []
  ############################################################################
  # 모범답안
  x_datas, y_datas = [], []
  for data, label in datas:
    x_datas.append(data)
    y_datas.append(label)
    
    if(len(x_datas) == batch_size):
      batches.append((x_datas, y_datas))
      x_datas, y_datas = [], []
#   batches.append(np.reshape(datas, (-1, 3)))
  

  ############################################################################

  return batches

<h2>FNN 모델 학습</h2>

<pre>
<b>1. read_file(file_path) 함수를 사용하여 학습 데이터 읽기</b>

<b>2. FNN 모델 객체 선언</b>

<b>3. epoch를 돌때마다 학습 데이터 셔플</b>

<b>3. batch 단위로 학습을 수행</b>
</pre>

In [0]:
import os
import numpy as np
from tqdm import tqdm

def train(flags):
  # 학습 데이터 읽기
  train_datas = read_file(flags["train_data_path"])

  # 모델 객체 선언
  model = FNN(flags)
  
  # tensorflow session 옵션 설정
  # allow_soft_placement=True : 어떤 device를 사용하여 연산할지 명시하지 않은 경우 자동으로 존재하는 디바이스 중에서 하나를 선택
  # gpu_options=tf.GPUOptions(allow_growth=True) : 연산 실행 과정에서 필요한만큼의 gpu 메모리만 사용
  sess_config = tf.ConfigProto(allow_soft_placement=True, 
                               gpu_options=tf.GPUOptions(allow_growth=True))

  # tensorflow를 실행하기 위한 session 
  with tf.Session(config=sess_config) as sess:
    # 그래프 초기화
    sess.run(tf.global_variables_initializer())
    # 학습 파일을 저장거나 불러오기 위한 saver 객체
    saver = tf.train.Saver(max_to_keep=10)          # 10개의 모델을 저장, 마지막이 최고가 아니라 중간에 나올수도 있으므로

    for epoch in tqdm(range(flags["epoch"])):
      # 학습 데이터 셔플
      np.random.shuffle(train_datas)
      # 학습 데이터를 batch 단위로 분할하여 저장
      batches = get_batch(train_datas, flags["batch_size"])

      losses = []
      # batch 단위로 학습을 진행하며 각 batch 별 loss를 구한다
      # batch 별 loss들의 평균을 구하여 이를 전체 데이터에 대한 loss로 사용
      for train_x, train_y in batches:
          loss, train_op = sess.run([model.loss, model.train_op],
                                    feed_dict={ model.inputs:train_x, 
                                               model.targets:train_y, 
                                               model.keep_prob:flags["keep_prob"] }
                                    )
          losses.append(loss)

      # 학습한 모델 파일 저장
      filename = os.path.join(flags["save_dir"], "model_{}.ckpt".format(epoch+1))
      saver.save(sess, filename)

      print("\tEpoch : {}, Average_Loss : {}".format(epoch+1, np.mean(losses)))

<h2>FNN 모델 평가</h2>

<pre>
<b>1. read_file(file_path) 함수를 사용하여 평가 데이터 읽기</b>

<b>2. FNN 모델 객체 선언</b>

<b>3. tf.train.Saver() 객체를 사용하여 학습한 모델 파일 중에서 가장 많이 학습된 파일로부터 가중치를 불러옴</b>
</pre>

In [0]:
import numpy as np
from tqdm import tqdm

def test(flags):
  # 평가 데이터 읽기
  test_datas = read_file(flags["test_data_path"])

  # 모델 객체 선언
  model = FNN(flags)
  # tensorflow session 옵션 설정
  sess_config = tf.ConfigProto(allow_soft_placement=True, 
                               gpu_options=tf.GPUOptions(allow_growth=True))

  # tensorflow를 실행하기 위한 session 
  with tf.Session(config=sess_config) as sess:
    # 그래프 초기화
    sess.run(tf.global_variables_initializer())
    # 학습 파일을 저장거나 불러오기 위한 saver 객체
    saver = tf.train.Saver()
    
    # 학습한 모델 파일 중에서 가장 많이 학습된 파일로부터 가중치를 불러옴
    print("Read from : " + str(tf.train.latest_checkpoint(flags["save_dir"])))
    saver.restore(sess, tf.train.latest_checkpoint(flags["save_dir"]))

    # 평가 데이터를 batch 단위로 분할하여 저장
    batches = get_batch(test_datas, flags["batch_size"])

    # 정답을 맞춘 개수
    correct_count = 0
    for test_x, test_y in tqdm(batches):
      predict_op = sess.run(model.predict_op,
                            feed_dict={ model.inputs:test_x, model.keep_prob:flags["keep_prob"] }
                            )

      # 모델의 outputs에는 각 클래스에 대한 분포가 저장되어 있고
      # np.argmax 함수를 통하여 가장 확률이 높은 클래스를 선택
      # 예시) 
      #  predict_op = [0,1, 0.3, 0.2] (각각 0일 확률, 1일 확률, 2일 확률)
      #  np.argmax(predict_op) = 1
      outputs, correct = np.argmax(predict_op, axis=-1), np.argmax(test_y, axis=-1)
      correct_count += np.sum(np.equal(outputs, correct))

    print("\nAccuracy : " + str(100.0*correct_count/(len(batches))))

<h2>모델의 hyper parameter 설정, 학습 및 평가 실행</h2>

<pre>
root_dir : 코드와 데이터가 있는 디렉토리 경로
save_dir : 학습한 모델 파일을 저장할 디렉토리 경로(디렉토리가 존재하지 않을 경우 자동으로 생성)

<b>flags : hyper parameter를 저장할 딕셔너리</b>
  flags.mode = 학습 또는 평가 설정("train" or "test")
  flags.save_dir = 학습한 모델 파일을 저장할 디렉토리 경로
  flags.batch_size = 한번에 학습할 데이터의 개수
  flags.epoch = 학습 횟수
  flags.learning_rate = 학습률
  flags.keep_prob = 노드를 보전할 확률
  flags.num_classes = 분류할 클래스의 개수(0~9, 따라서 10개)
  flags.train_data_path = 학습데이터 파일 경로
  flags.test_data_path = 평가데이터 파일 경로

<b>mode 별 hyper parameter 변경</b>
  학습하는 경우 : mode를 "train"으로 설정, 나머지는 기본 설정 그대로 유지
  평가하는 경우 : mode를 "test"로, batch_size는 1로, keep_prob은 1.0으로 변경
</pre>

In [7]:
import os

if __name__ == "__main__":
  root_dir = "/gdrive/My Drive/colab/week 6"
  save_dir = os.path.join(root_dir, "model")
  if not os.path.exists(save_dir):
      os.makedirs(save_dir)

  flags = {"mode":"train",
           "save_dir":save_dir,
           "batch_size":64,
           "epoch":10,
           "learning_rate":0.001,
           "keep_prob":0.7,
           "num_classes":10,
           "train_data_path":os.path.join(root_dir, "mnist_train.csv"),
           "test_data_path":os.path.join(root_dir, "mnist_test.csv")          
          }
  
  tf.reset_default_graph()
  if(flags["mode"] == "train"):
      train(flags)
  elif(flags["mode"] == "test"):
      flags["batch_size"] = 1  
      flags["keep_prob"] = 1.0
      test(flags)
  else:
      print("Unknown mode")
      exit(0)

 10%|█         | 1/10 [00:07<01:05,  7.32s/it]

	Epoch : 1, Average_Loss : 18.32221794128418


 20%|██        | 2/10 [00:13<00:56,  7.03s/it]

	Epoch : 2, Average_Loss : 6.807912349700928


 30%|███       | 3/10 [00:20<00:48,  6.90s/it]

...

	Epoch : 8, Average_Loss : 3.600595235824585


 90%|█████████ | 9/10 [00:59<00:06,  6.52s/it]

	Epoch : 9, Average_Loss : 3.439873695373535


100%|██████████| 10/10 [01:05<00:00,  6.57s/it]

	Epoch : 10, Average_Loss : 3.246338367462158


In [8]:
import os

if __name__ == "__main__":
  root_dir = "/gdrive/My Drive/colab/week 6"
  save_dir = os.path.join(root_dir, "model")
  if not os.path.exists(save_dir):
      os.makedirs(save_dir)

  flags = {"mode":"test",
           "save_dir":save_dir,
           "batch_size":64,
           "epoch":10,
           "learning_rate":0.001,
           "keep_prob":0.7,
           "num_classes":10,
           "train_data_path":os.path.join(root_dir, "mnist_train.csv"),
           "test_data_path":os.path.join(root_dir, "mnist_test.csv")          
          }
  
  tf.reset_default_graph()
  if(flags["mode"] == "train"):
      train(flags)
  elif(flags["mode"] == "test"):
      flags["batch_size"] = 1  
      flags["keep_prob"] = 1.0
      test(flags)
  else:
      print("Unknown mode")
      exit(0)

  0%|          | 0/10000 [00:00<?, ?it/s]

Read from : /gdrive/My Drive/colab/week 6/model/model_10.ckpt


100%|██████████| 10000/10000 [00:13<00:00, 750.37it/s]



Accuracy : 86.72
